In [ ]:
import numpy as np
import random
import copy
import time
from tabulate import tabulate
from abc import ABC, abstractmethod
from typing import Any, Callable

class CheckersGame:
    EMPTY = 0
    USER_PIECE = 1
    USER_KING = 2
    BOT_PIECE = 3
    BOT_KING = 4
    USER_DIRECTIONS = [(1, -1), (1, 1)]
    BOT_DIRECTIONS = [(-1, -1), (-1, 1)]
    KING_DIRECTIONS = USER_DIRECTIONS + BOT_DIRECTIONS

    def __init__(self):
        self.board = self.initialize_board()
        self.user_turn = True
        self.is_endgame = False
        self.king_chase = False

    #checked
    def initialize_board(self):
        """
        Initializes the checkers board with pieces in their starting positions.
        """
        board = np.zeros((8, 8), dtype=int)
        for row in range(3):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row, col] = self.USER_PIECE
        for row in range(5, 8):
            for col in range(8):
                if (row + col) % 2 == 1:
                    board[row, col] = self.BOT_PIECE
        return board

    #checked
    def display_board(self):
        """
        Display the board in a cleaner format with Unicode symbols for pieces.
        """
        symbols = {
            self.EMPTY: " ",
            self.USER_PIECE: "●",
            self.USER_KING: "♚",
            self.BOT_PIECE: "○",
            self.BOT_KING: "♔",
        }

        # Print column headers
        print("    " + " ".join(map(str, range(8))))
        print("   " + "-" * 17)

        for r in range(7,-1,-1):
            row = " ".join(symbols[self.board[r, c]] for c in range(8))
            print(f"{r} | {row}")
        print()

    def move_piece(self, start, end):
        """
        Moves a piece from the start position to the end position.
        Handles captures, multi-jumps, and king promotion correctly.
        """
        piece = self.board[start]

        # Clear the starting position
        self.board[start] = self.EMPTY

        # Check for king promotion before moving the piece
        if piece == self.USER_PIECE and end[0] == 7:
            piece = self.USER_KING
        elif piece == self.BOT_PIECE and end[0] == 0:
            piece = self.BOT_KING

        # Update the destination position
        self.board[end] = piece

        # Check for captures
        if abs(start[0] - end[0]) == 2:  # A jump occurred
            mid_r, mid_c = (start[0] + end[0]) // 2, (start[1] + end[1]) // 2
            self.board[mid_r, mid_c] = self.EMPTY

            # Check for additional jumps
            _, further_captures = self.get_valid_moves(piece, end)
            if further_captures:  # If multi-jumps are possible
                next_jump = further_captures[0]
                self.move_piece(end, next_jump)

    def is_game_over(self):
        """
        Checks if the game is over (no valid moves for either player).
        """
        user_moves = self.get_all_valid_moves("user")
        bot_moves = self.get_all_valid_moves("bot")
        return len(user_moves) == 0 or len(bot_moves) == 0

    def get_valid_moves(self, piece, position, visited=None):
        """
        Returns valid moves (both captures and regular moves) for a specific piece at a given position.
        """
        if visited is None:
            visited = set()

        moves = []
        captures = []
        directions = (
            self.KING_DIRECTIONS
            if piece in [self.USER_KING, self.BOT_KING]
            else self.USER_DIRECTIONS
            if piece == self.USER_PIECE
            else self.BOT_DIRECTIONS
        )

        for dr, dc in directions:
            r, c = position[0] + dr, position[1] + dc

            # Regular move
            if 0 <= r < 8 and 0 <= c < 8 and self.board[r, c] == self.EMPTY:
                moves.append((r, c))
            # Capture move
            elif (
                0 <= r + dr < 8
                and 0 <= c + dc < 8
                and self.board[r, c] != self.EMPTY
                and (r, c) not in visited
                and (
                    (piece in [self.USER_PIECE, self.USER_KING] and self.board[r, c] in [self.BOT_PIECE, self.BOT_KING])
                    or (piece in [self.BOT_PIECE, self.BOT_KING] and self.board[r, c] in [self.USER_PIECE, self.USER_KING])
                )
                and self.board[r + dr, c + dc] == self.EMPTY
            ):
                captures.append((r + dr, c + dc))
                # Recursively explore multi-jumps
                sub_captures = self.get_valid_moves(piece, (r + dr, c + dc), visited | {(r, c)})
                for sub_capture in sub_captures[1]:
                    captures.append(sub_capture)

        return moves, captures

    def get_all_valid_moves(self, player):
        moves = []
        captures = []

        for r in range(8):
            for c in range(8):
                piece = self.board[r, c]
                if (
                    (player == "user" and piece in [self.USER_PIECE, self.USER_KING])
                    or (player == "bot" and piece in [self.BOT_PIECE, self.BOT_KING])
                ):
                    pos_moves, pos_captures = self.get_valid_moves(piece, (r, c))
                    moves.extend([(r, c, end) for end in pos_moves])
                    captures.extend([(r, c, end) for end in pos_captures])

        return moves + captures

    def get_reward(self, board):
        """
        #Computes the reward for the given board state.
        #Penalizes repetitive moves and rewards diversity and progress.
        """

        reward = 0

        POSITION_PENALTY = -3
        POSITION_REWARD = 5
        KING_BONUS = 10
        PROMOTION_BONUS = 7
        EDGE_BONUS = 4
        CENTER_BONUS = 2
        VULNERABLE_PENALTY = -20
        POTENTIAL_CAPTURE_REWARD = 15

        if self.is_endgame:
            POSITION_PENALTY -= 6
            KING_BONUS -= 7
            EDGE_BONUS -= 2
            CENTER_BONUS += 8
            VULNERABLE_PENALTY = -20
            POTENTIAL_CAPTURE_REWARD = 40
        elif self.king_chase:
            PROMOTION_BONUS += 13
            EDGE_BONUS -= 4
            CENTER_BONUS -= 2
            POTENTIAL_CAPTURE_REWARD -= 10


        center_positions = [(3, 3), (3, 4), (4, 3), (4, 4)]  # Center 2x2 square

        for row in range(8):
            for col in range(8):
                piece = board[row, col]

                #bot pieces
                if piece == self.BOT_PIECE:
                    reward += POSITION_REWARD
                    if row == 7:
                        reward += PROMOTION_BONUS

                    reward += EDGE_BONUS - abs(row/2 - 3.5) - abs(col/2 - 3.5)

                    #vulnerability
                    for dr, dc in self.KING_DIRECTIONS:
                      enemy_row, enemy_col = row + dr, col + dc
                      jump_row, jump_col = row - dr, col - dc
                      if (
                          0 <= enemy_row < 8 and 0 <= enemy_col < 8 and
                          0 <= jump_row < 8 and 0 <= jump_col < 8 and
                          board[enemy_row, enemy_col] in [self.USER_PIECE, self.USER_KING] and
                          board[jump_row, jump_col] == 0
                      ):
                          reward += VULNERABLE_PENALTY
                          break
                #bot king
                elif piece == self.BOT_KING:
                    reward += KING_BONUS

                    reward += CENTER_BONUS - abs(row - 3.5) - abs(col - 3.5)

                    for dr, dc in self.KING_DIRECTIONS:
                        enemy_row, enemy_col = row + 2*dr, col + 2*dc
                        jump_row, jump_col = row - 2*dr, col - 2*dc
                        if (
                            0 <= enemy_row < 8 and 0 <= enemy_col < 8 and
                            0 <= jump_row < 8 and 0 <= jump_col < 8 and
                            board[enemy_row, enemy_col] in [self.USER_PIECE, self.USER_KING] and
                            board[jump_row, jump_col] == 0
                        ):
                            reward += POTENTIAL_CAPTURE_REWARD
                            break

                    #vulnerability
                    for dr, dc in self.KING_DIRECTIONS:
                        enemy_row, enemy_col = row + dr, col + dc
                        jump_row, jump_col = row - dr, col - dc
                        if (
                            0 <= enemy_row < 8 and 0 <= enemy_col < 8 and
                            0 <= jump_row < 8 and 0 <= jump_col < 8 and
                            board[enemy_row, enemy_col] in [self.USER_PIECE, self.USER_KING] and
                            board[jump_row, jump_col] == 0
                        ):
                            reward += VULNERABLE_PENALTY
                            break

                #user piece
                elif piece == self.USER_PIECE:
                    reward -= POSITION_REWARD
                    if row == 0:
                        reward -= PROMOTION_BONUS

                #user bot
                elif piece == self.USER_KING:
                    reward -= KING_BONUS

        return reward


In [2]:
class MDP():
    """
    Data structure for a Markov Decision Process. In mathematical terms,
    MDPs are sometimes defined in terms of a tuple consisting of the various
    components of the MDP, written (S, A, T, R, gamma):

    gamma: discount factor
    S: state space
    A: action space
    T: transition function
    R: reward function
    TR: sample transition and reward. We will us `TR` later to sample the next
        state and reward given the current state and action: s_prime, r = TR(s, a)
    """
    def __init__(self,
                 gamma: float,
                 S: list[Any],
                 A: list[Any],
                 T: Callable[[Any, Any, Any], float] | np.ndarray,
                 R: Callable[[Any, Any], float] | np.ndarray,
                 TR: Callable[[Any, Any], tuple[Any, float]] = None):
        self.gamma = gamma  # discount factor
        self.S = S          # state space
        self.A = A          # action space

        # reward function R(s, a)
        if type(R) == np.ndarray:
            self.R = lambda s, a: R[s, a]
        else:
            self.R = R

        # transition function T(s, a, s')
        # sample next state and reward given current state and action: s', r = TR(s, a)
        if type(T) == np.ndarray:
            self.T = lambda s, a, s_prime: T[s, a, s_prime]
            self.TR = lambda s, a: (np.random.choice(len(self.S), p=T[s, a]), self.R(s, a)) if not np.all(T[s, a] == 0) else (np.random.choice(len(self.S)), self.R(s, a))
        else:
            self.T = T
            self.TR = TR

    def lookahead(self, U: Callable[[Any], float] | np.ndarray, s: Any, a: Any) -> float:
        if callable(U):
            return self.R(s, a) + self.gamma * np.sum([self.T(s, a, s_prime) * U(s_prime) for s_prime in self.S])
        return self.R(s, a) + self.gamma * np.sum([self.T(s, a, s_prime) * U[i] for i, s_prime in enumerate(self.S)])

    def iterative_policy_evaluation(self, policy: Callable[[Any], Any], k_max: int) -> np.ndarray:
        U = np.zeros(len(self.S))
        for _ in range(k_max):
            U = np.array([self.lookahead(U, s, policy(s)) for s in self.S])
        return U

    def policy_evaluation(self, policy: Callable[[Any], Any]) -> np.ndarray:
        R_prime = np.array([self.R(s, policy(s)) for s in self.S])
        T_prime = np.array([[self.T(s, policy(s), s_prime) for s_prime in self.S] for s in self.S])
        I = np.eye(len(self.S))
        return np.linalg.solve(I - self.gamma * T_prime, R_prime)

    def greedy(self, U: Callable[[Any], float] | np.ndarray, s: Any) -> tuple[float, Any]:
        expected_rewards = [self.lookahead(U, s, a) for a in self.A]
        idx = np.argmax(expected_rewards)
        return self.A[idx], expected_rewards[idx]

    def backup(self, U: Callable[[Any], float] | np.ndarray, s: Any) -> float:
        return np.max([self.lookahead(U, s, a) for a in self.A])

    def randstep(self, s: Any, a: Any) -> tuple[Any, float]:
        return self.TR(s, a)

    def simulate(self, s: Any, policy: Callable[[Any], Any], d: int) -> list[tuple[Any, Any, float]]:
        trajectory = []
        for _ in range(d):
            a = policy(s)
            s_prime, r = self.TR(s, a)
            trajectory.append((s, a, r))
            s = s_prime
        return trajectory

    def random_policy(self):
        return lambda s, A=self.A: random.choices(A)[0]

class MDPSolutionMethod(ABC):
    pass

class OnlinePlanningMethod(MDPSolutionMethod):
    @abstractmethod
    def __call__(self, s: Any) -> Any:
        pass

class MonteCarloTreeSearch(OnlinePlanningMethod):
    def __init__(self,
                 P: MDP,
                 N: dict[tuple[Any, Any], int],
                 Q: dict[tuple[Any, Any], float],
                 d: int,
                 m: int,
                 c: float,
                 U: Callable[[Any], float]):
        self.P = P  # problem
        self.N = N  # visit counts
        self.Q = Q  # action value estimates
        self.d = d  # depth
        self.m = m  # number of simulations
        self.c = c  # exploration constant
        self.U = U  # value function estimate

    def __call__(self, s: Any) -> Any:
        for _ in range(self.m):
            self.simulate(s, d=self.d)
        rewards = np.array([self.Q[(s, a)] for a in self.P.A])/2
        print(f"Rewards of new possible states: {rewards}")
        return self.P.A[np.argmax(rewards)]

    def simulate(self, s: Any, d: int):
        if d <= 0:
            return self.U(s)
        if (s, self.P.A[0]) not in self.N:
            for a in self.P.A:
                self.N[(s, a)] = 0
                self.Q[(s, a)] = 0.0
            return self.U(s)
        a = self.explore(s)
        s_prime, r = self.P.randstep(s, a)
        q = r + self.P.gamma * self.simulate(s_prime, d - 1)
        self.N[(s, a)] += 1
        self.Q[(s, a)] += (q - self.Q[(s, a)]) / self.N[(s, a)]
        return q

    def explore(self, s: Any) -> Any:
        A, N = self.P.A, self.N
        Ns = np.sum([N[(s, a)] for a in A])
        return A[np.argmax([self.ucb1(s, a, Ns) for a in A])]

    def ucb1(self, s: Any, a: Any, Ns: int) -> float:
        N, Q, c = self.N, self.Q, self.c
        return Q[(s, a)] + c*self.bonus(N[(s, a)], Ns)

    @staticmethod
    def bonus(Nsa: int, Ns: int) -> float:
        return np.inf if Nsa == 0 else np.sqrt(np.log(Ns)/Nsa)

In [3]:
class MCTSBot:
    def __init__(self, checkers_game, mcts_params):
        """
        Initialize the MCTSBot with the CheckersGame and MCTS parameters.

        Args:
            checkers_game (CheckersGame): The current checkers game instance.
            mcts_params (dict): Parameters for MCTS including depth, exploration constant, etc.
        """
        self.game = checkers_game
        self.mcts_params = mcts_params  # Contains depth, exploration constant, number of simulations, etc.

    def get_best_move(self):
        """
        Run Monte Carlo Tree Search to find the best move for the bot.

        Returns:
            best_state (CheckersGame): The new game state (CheckersGame object) after the best move.
        """
        # Define the MDP for MCTS
        initial_state = copy.deepcopy(self.game)
        valid_moves = self.game.get_all_valid_moves("bot")

        # Create an MDP representation
        mdp = MDP(
            gamma=1.0,  # No discounting, as we are planning one step at a time
            S=[copy.deepcopy(self.game)],
            A=valid_moves,
            T=self.transition_function,
            R=self.reward_function,
            TR=self.sample_transition
        )

        # Initialize visit counts and action value estimates
        N = {}
        Q = {}

        # Create the MCTS instance
        mcts = MonteCarloTreeSearch(
            P=mdp,
            N=N,
            Q=Q,
            d=self.mcts_params["depth"],
            m=self.mcts_params["simulations"],
            c=self.mcts_params["exploration_constant"],
            U=self.utility_function
        )

        # Perform MCTS to determine the best action
        best_action = mcts(copy.deepcopy(initial_state))

        # Create a new game state based on the best action
        best_state = self.apply_action(copy.deepcopy(initial_state), best_action)

        return best_state

    def get_move_from_states(self, old_state, new_state):
        """
        Determines the bot's move and captures based on the change in board state.

        Args:
            old_state (CheckersGame): The state of the game before the move.
            new_state (CheckersGame): The state of the game after the move.

        Returns:
            dict: A dictionary containing the bot's move and captured pieces:
                {
                    "original_position": (row, col),
                    "new_position": (row, col),
                    "captured_positions": [(row, col), ...]
                }
        """
        move = {
            "original_position": None,
            "new_position": None,
            "captured_positions": []
        }

        # Compare old and new boards to identify the moved piece and captures
        for r in range(8):
            for c in range(8):
                # Detect original position (piece moved from here)
                if old_state.board[r, c] in [CheckersGame.BOT_PIECE, CheckersGame.BOT_KING] and new_state.board[r, c] == CheckersGame.EMPTY:
                    move["original_position"] = (r, c)
                # Detect new position (piece moved to here)
                elif old_state.board[r, c] == CheckersGame.EMPTY and new_state.board[r, c] in [CheckersGame.BOT_PIECE, CheckersGame.BOT_KING]:
                    move["new_position"] = (r, c)
                # Detect captured pieces (user pieces removed)
                elif old_state.board[r, c] in [CheckersGame.USER_PIECE, CheckersGame.USER_KING] and new_state.board[r, c] == CheckersGame.EMPTY:
                    move["captured_positions"].append((r, c))

        # Validate the move to avoid returning None
        if move["original_position"] is None or move["new_position"] is None:
            raise ValueError("Move tracking failed: Original or new position not identified. Check board states.")

        return move


    def transition_function(self, state, action, next_state):
        """
        Transition function for MCTS. Checks whether a given action transitions to a valid next state.

        Args:
            state (CheckersGame): Current game state.
            action (tuple): Move (start_row, start_col, end_row, end_col).
            next_state (CheckersGame): Next game state.

        Returns:
            float: 1 if valid transition, 0 otherwise.
        """
        temp_state = copy.deepcopy(state)
        start, end = (action[0], action[1]), (action[2][0], action[2][1])
        temp_state.move_piece(start, end)
        return 1.0 if np.array_equal(temp_state.board, next_state.board) else 0.0

    def reward_function(self, state, action):
        """
        Reward function for MCTS based on the bot's position after making the action.

        Args:
            state (CheckersGame): Current game state.
            action (tuple): Move (start_row, start_col, end_row, end_col).

        Returns:
            float: Reward value for the resulting state.
        """
        temp_state = copy.deepcopy(state)
        start, end = (action[0], action[1]), (action[2][0], action[2][1])
        temp_state.move_piece(start, end)
        return temp_state.get_reward(temp_state.board)

    def sample_transition(self, state, action):
        """
        Samples the next state and reward for MCTS given a state and action.

        Args:
            state (CheckersGame): Current game state.
            action (tuple): Move (start_row, start_col, end_row, end_col).

        Returns:
            tuple: (next_state, reward)
        """
        temp_state = copy.deepcopy(state)
        start, end = (action[0], action[1]), (action[2][0], action[2][1])
        temp_state.move_piece(start, end)
        reward = temp_state.get_reward(temp_state.board)
        return temp_state, reward

    def utility_function(self, state):
        """
        Utility function for evaluating the value of a state.

        Args:
            state (CheckersGame): Current game state.

        Returns:
            float: Utility value of the state.
        """
        return state.get_reward(state.board)

    def apply_action(self, state, action):
        """
        Applies a given action to a state and returns the resulting state.

        Args:
            state (CheckersGame): Current game state.
            action (tuple): Move (start_row, start_col, end_row, end_col).

        Returns:
            CheckersGame: The updated game state.
        """
        start, end = (action[0], action[1]), (action[2][0], action[2][1])
        state.move_piece(start, end)
        return state

In [4]:
def play_checkers(num_players):
    game = CheckersGame()
    bot = MCTSBot(game, mcts_params={"depth": 3, "simulations": 1000, "exploration_constant": 2})
    print("Welcome to Checkers! You are Player 1 (● - ♚). The bot is Player 2 (○ - ♔).\n")
    game.display_board()
    states = []

    while not game.is_game_over():
        if game.user_turn:
            print("Your Turn!")
            user_moves = game.get_all_valid_moves("user")
            if not user_moves:
                print("No valid moves available. You lose!")
                print("Bot Wins!")
                return

            # Display possible moves
            for i, move in enumerate(user_moves):
                print(f"{i}: Move from {move[:2]} to {move[2]}")

            # Loop until a valid index is entered
            selected_move = None
            while selected_move is None:
                try:
                    move_idx = int(input("Select your move (index): "))
                    if 0 <= move_idx < len(user_moves):  # Validate index
                        selected_move = user_moves[move_idx]
                    else:
                        print(f"Invalid input! Please enter a number between 0 and {len(user_moves) - 1}.")
                except ValueError:
                    print("Invalid input! Please enter a valid integer.")

            # Execute the selected move
            if selected_move:
                game.move_piece(selected_move[:2], selected_move[2])
        else:
            print("Bot's Turn...")

            if num_players == 1:
                start_time = time.time()
                best_state = bot.get_best_move()
                move = bot.get_move_from_states(game, best_state)
                game.board = best_state.board
                print(f"Reward for new state: {game.get_reward(game.board)}")

                # Log the bot's move
                print(f"Bot moved from {move['original_position']} to {move['new_position']} in {time.time() - start_time:.2f} seconds.")
                if move['captured_positions']:
                    print(f"Bot captured pieces at {move['captured_positions']}.")

            else:
                bot_moves = game.get_all_valid_moves("bot")
                if not bot_moves:
                    print("No valid moves available. Bot loses!")
                    print("You Win!")
                    return

                # Display possible moves
                for i, move in enumerate(bot_moves):
                    print(f"{i}: Move from {move[:2]} to {move[2]}")

                # Loop until a valid index is entered
                selected_move = None
                while selected_move is None:
                    try:
                        move_idx = int(input("Select your move (index): "))
                        if 0 <= move_idx < len(bot_moves):  # Validate index
                            selected_move = bot_moves[move_idx]
                        else:
                            print(f"Invalid input! Please enter a number between 0 and {len(bot_moves) - 1}.")
                    except ValueError:
                        print("Invalid input! Please enter a valid integer.")

                # Execute the selected move
                if selected_move:
                    game.move_piece(selected_move[:2], selected_move[2])

            num_user_pieces = np.count_nonzero(game.board == game.USER_PIECE)
            num_user_kings = np.count_nonzero(game.board == game.USER_KING)
            num_bot_pieces = np.count_nonzero(game.board == game.BOT_PIECE)
            num_bot_kings = np.count_nonzero(game.board == game.BOT_KING)
            num_total = num_bot_kings + num_bot_pieces + num_user_kings + num_user_pieces

            if (num_total <= 8 or
                (num_bot_kings+num_bot_pieces)/(num_user_kings+num_user_pieces+1e-4)>2 or
                (num_bot_kings/(num_user_kings+1e-4))>3):
                game.is_endgame = True
            else:
                game.is_endgame = False

            if (game.is_endgame and num_bot_kings<=2):
                game.king_chase = True
            else:
                game.king_chase = False



        # Display the updated board and switch turns
        game.display_board()
        game.user_turn = not game.user_turn  # Switch turns
        cur_game = copy.deepcopy(game)
        states.append(cur_game)

    print("Game Over!")
    # Determine winner
    if game.get_all_valid_moves("user"):
        print("You Win!")
    else:
        print("Bot Wins!")

    val = input("Would you like to print all states? (y/n)")
    if val == "y":
        print("All states:")
        for state in states:
            state.display_board()


In [5]:
num_players = 1
play_checkers(num_players)

Welcome to Checkers! You are Player 1 (● - ♚). The bot is Player 2 (○ - ♔).

    0 1 2 3 4 5 6 7
   -----------------
7 | ○   ○   ○   ○  
6 |   ○   ○   ○   ○
5 | ○   ○   ○   ○  
4 |                
3 |                
2 |   ●   ●   ●   ●
1 | ●   ●   ●   ●  
0 |   ●   ●   ●   ●

Your Turn!
0: Move from (2, 1) to (3, 0)
1: Move from (2, 1) to (3, 2)
2: Move from (2, 3) to (3, 2)
3: Move from (2, 3) to (3, 4)
4: Move from (2, 5) to (3, 4)
5: Move from (2, 5) to (3, 6)
6: Move from (2, 7) to (3, 6)
Select your move (index): 6
    0 1 2 3 4 5 6 7
   -----------------
7 | ○   ○   ○   ○  
6 |   ○   ○   ○   ○
5 | ○   ○   ○   ○  
4 |                
3 |             ●  
2 |   ●   ●   ●    
1 | ●   ●   ●   ●  
0 |   ●   ●   ●   ●

Bot's Turn...
Rewards of new possible states: [20. 19. 20. 19.  0. 19. 20.]
Reward for new state: 20.0
Bot moved from (5, 0) to (4, 1) in 0.30 seconds.
    0 1 2 3 4 5 6 7
   -----------------
7 | ○   ○   ○   ○  
6 |   ○   ○   ○   ○
5 |     ○   ○   ○  
4 |   ○          

KeyboardInterrupt: Interrupted by user